In [2]:
#%pip install requests-html requests pandas

In [5]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import time

In [2]:
def BaseLinks(tag, totPageNum):
    base_url = 'https://stackoverflow.com/questions/tagged/'
    query_filter = "Votes"
    pageLinks = []
    
    for pageNum in range (1, totPageNum+1):
        url = f"{base_url}{tag}?tab={query_filter}&page={pageNum}&pagesize=50"
        pageLinks.append(url)
    
    return pageLinks

In [3]:
#asdf = BaseLinks('pytorch', 384 )
#

In [4]:
def ParseQLinksFromBase(pageLinks):
    
    href = []
    
    for url in pageLinks:
        time.sleep(5)
        r = requests.get(url)
        so_soup = BeautifulSoup(r.text, 'html.parser')
        question_block = so_soup.find(id = "questions")
        if (question_block == None):
            print("Error in URL: "+ url)
        else:
            question_links = question_block.find_all("a", class_="s-link",href=True)

        for qlink in question_links:
            href.append(qlink['href'])

    return href


In [17]:
pageLinks = BaseLinks('pytorch', 385)
len(pageLinks)
#print(pageLinks)

385

In [18]:
pytorchQLinks = ParseQLinksFromBase(pageLinks)
len(pytorchQLinks)

19209

In [29]:
qID = []
for i in range(len(pytorchQLinks)):
    qID.append(pytorchQLinks[i].split("/")[2])

In [30]:
qID

['48152674',
 '42703500',
 '42479902',
 '48001598',
 '42480111',
 '49433936',
 '49643225',
 '60018578',
 '48915810',
 '51433378',
 '51030782',
 '49201236',
 '53975717',
 '55266154',
 '43451125',
 '59013109',
 '49768306',
 '56741087',
 '50999977',
 '56859803',
 '59129812',
 '50544730',
 '48302810',
 '57237352',
 '54307225',
 '50935345',
 '42704283',
 '53903373',
 '44429199',
 '44524901',
 '52468956',
 '57727372',
 '54716377',
 '50307707',
 '53580088',
 '51677788',
 '48324152',
 '46826218',
 '53998282',
 '46774641',
 '51691563',
 '54374935',
 '43328632',
 '41924453',
 '53266350',
 '53623472',
 '56816241',
 '53419474',
 '50747947',
 '55627780',
 '49390842',
 '47863001',
 '48482787',
 '58151507',
 '45113245',
 '54216920',
 '44732217',
 '62359175',
 '54843067',
 '55322434',
 '53900396',
 '48158017',
 '51748138',
 '49710537',
 '51911749',
 '49036993',
 '53375422',
 '44328530',
 '63582590',
 '51743214',
 '60987997',
 '44371560',
 '48274929',
 '54527439',
 '52946920',
 '53994625',
 '44790670',

In [31]:
import csv

In [32]:
colHeader = ['QID', 'QLink']
rows = []
with open('PostLinks.csv','w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(colHeader)
    writer.writerows(zip(qID,pytorchQLinks))

In [52]:
def getContentData(contentUrl):
    r = requests.get(contentUrl)
    content_soup = BeautifulSoup(r.text, 'html.parser')
    qID = contentUrl.split("/")[4]
    qHeader = content_soup.find(class_ = "question-hyperlink").text
    #mainbar = content_soup.find(id = "mainbar")
    qvote = content_soup.find(class_ = "question js-question").attrs.get('data-score')
    question = content_soup.find(class_ = "s-prose js-post-body").text
    qCodePart = content_soup.find(class_ = "s-prose js-post-body").findAll("code")
    qPostTime = content_soup.find(class_ = "post-signature owner flex--item").find('span').attrs.get('title')
    qAuthor = content_soup.find(class_ = "post-signature owner flex--item").find(class_ = "user-details").find("a").text
    qAuthorRep = content_soup.find(class_ = "post-signature owner flex--item").find(class_ = "user-details").find(class_ = 'reputation-score').text
    #qComments = content_soup.find(class_ = ) comments are not collected
    numAnswers = content_soup.find(id = 'answers').find(class_ = "mb0").get("data-answercount")
    hasAcceptedAns = content_soup.find(class_ = "answer js-answer accepted-answer js-accepted-answer")
    accAns = accAnsCode = ""

    if (hasAcceptedAns != None):
        accAns = content_soup.find(class_ = "answer js-answer accepted-answer js-accepted-answer").find(class_ = 's-prose js-post-body').text
        accAnsCode = content_soup.find(class_ = "answer js-answer accepted-answer js-accepted-answer").find(class_ = 's-prose js-post-body').findAll("code")
        numAnswers -= 1

    allAnsIds = []
    allAnswers = []
    allAnswersCode = []
    answerBlock = content_soup.findAll(class_ = "answer js-answer")
    for i in range(int(numAnswers)):
        singleBlock = answerBlock[i]
        ansId = singleBlock.attrs.get("data-answerid")
        allAnsIds.append(ansId)
        ans = singleBlock.find(class_ = "s-prose js-post-body").text
        allAnswers.append(ans)
        ansCode = singleBlock.find(class_ = "s-prose js-post-body").findAll("code")
        allAnswersCode.append(ansCode)

        #colHeader = [qID, qHeader, qvote, question, qCodePart, qTime, qAuthor, qAuthorRep, numAnswers, allAnswers, allAnswersCode]
    rowdata = [qID, qHeader, qvote, question, qCodePart, qPostTime, qAuthor, qAuthorRep, numAnswers, accAns, accAnsCode, allAnsIds, allAnswers, allAnswersCode]

    return rowdata


In [53]:
temp = getContentData('https://stackoverflow.com/questions/46833212/looping-through-elements-with-beautifulsoup')

In [41]:
#print(temp)
#temp[0]

In [48]:
columnNames = ["qID", "qHeader", "qvote", "question", "qCodePart", 'qPostTime', "qAuthor", 'qAuthorRep', 'numAnswers', "accAns", "accAnsCode", "allAnsIds", "allAnswers", "allAnswersCode"]


In [58]:
df = pd.DataFrame(columns=columnNames)

In [59]:
df

,qID,qHeader,qvote,question,qCodePart,qPostTime,qAuthor,qAuthorRep,numAnswers,accAns,accAnsCode,allAnsIds,allAnswers,allAnswersCode


In [57]:
df.append(temp, ignore_index=False)

,0,accAns,accAnsCode,allAnsIds,allAnswers,allAnswersCode,numAnswers,qAuthor,qAuthorRep,qCodePart,qHeader,qID,qPostTime,question,qvote
0,46833212,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Looping through elements with Beautifulsoup,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,\nI'm trying to store some data that's scraped...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,[[# parse the page and store in var soup\nsoup...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2017-10-19 15:12:28Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Rhys Edwards,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,751,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
